# Movie Recommender System with SVD 
## 1. Loading the Data

In [ ]:
import pandas as pd
import numpy as np

ratings_list = [i.strip().split("::") for i in open('./ml-1m/ratings.dat', 'r').readlines()]
users_list = [i.strip().split("::") for i in open('./ml-1m/users.dat', 'r').readlines()]
movies_list = [i.strip().split("::") for i in open('./ml-1m/movies.dat', 'r',encoding = "ISO-8859-1").readlines()]

print(ratings_list[0])
print(users_list[0])
print(movies_list[0])

## - Data type conversion to numpy array and pandas DataFrame

In [ ]:
ratings = np.array(ratings_list)
users = np.array(users_list)
movies = np.array(movies_list)

In [ ]:
ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])

#convert string data type to int64 
movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)

### * check dataframe

In [ ]:
movies_df.head()

In [ ]:
ratings_df.head()

## 2. Make pivot table
* 피봇 테이블(pivot table) : 데이터 열 중에서 두 개를 키(key)로 사용하여 데이터를 선택하는 방법을 말한다.
* 첫번째 인수로는 행 인덱스로 사용할 열 이름, 두번째 인수로는 열 인덱스로 사용할 열 이름, 그리고 마지막으로 데이터로 사용할 열 이름을 넣는다.
* fillna: Fill NA/NaN values using the specified method

In [ ]:
"""
=====================
Fill in the cell !
=====================

* Useful functions:
    - DataFrame.pivot(index, columns, values)
    - DataFrame.fillna(value)
    
* Step by step
    1. Make pivot table "R_df" with rating DataFrame
    2. fill the NaN values
"""


## 3. Normalize by each users mean convert it from a dataframe to a numpy array

In [ ]:
# DataFrame type to matrix
R = R_df.as_matrix()

"""
=====================
Fill in the cell !
=====================

* Useful functions:
    - np.mean(a, axis), reshape(-1,1) 
    
* Step by step
    1. make "user_ratings_mean" variable 
        : compute rating mean of each user
        (caution! matrix shape : [number of users, 1] , matrix dim : 2 dim)
    2. make "R_normalized" variable
        : subtract that matrix from "R"
        
"""


## 4. Singular Value Decomposition

In [ ]:
from scipy.sparse.linalg import svds
"""
=====================
Fill in the cell !
=====================

* Useful functions:
    - svds(A, k) : return U, sigma, Vt
    - np.diag(v) : Extract a diagonal matrix
    
* Step by step
    1. make "U, sigma, Vt" variables for svds funtion's output (k = 50)
    2. Diagonalize the sigma value 

"""


## 5. Making Predictions 

* multiply $U$, $\Sigma$, and $V^{T}$ back to get the rank $k=50$ approximation of $R$.

* add the user means back to get the actual star ratings prediction.

In [ ]:
"""
=====================
Fill in the cell !
=====================

* Useful functions:
    - np.dot
* Step by step
    1. make "all_user_predicted_ratings" variable for predictions
        1) multiply each variables 
        2) add user ratings mean again
"""


## 6. Making Movie Recommendations

In [ ]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
preds_df.head()

In [ ]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    #print(user_data.head())
    
    # left: use only keys from left frame, similar to a SQL left outer join; preserve key order
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )
    #print(user_full.head())
    
    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print ('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    # remove already rated movie row
    recommendations = movies_df[~movies_df["MovieID"].isin(user_full['MovieID'])]
    
    # merge with prediction and movie information
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID')
    
    #rename userID column to prediction
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'})
    
    #sorting prediction value to descending order and 
    recommendations = recommendations.sort_values(['Predictions'], ascending = False)
    recommendations = recommendations.iloc[:num_recommendations, :-1]

    return user_full, recommendations


In [ ]:
already_rated, predictions = recommend_movies(preds_df, 837, movies_df, ratings_df, 10)

In [ ]:
already_rated.head(10)

In [ ]:
predictions